# Librerias

In [1]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
from mtbpy import mtbpy

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

## Base de Datos

In [2]:
df = pd.read_csv('conecta.csv')

## Tokenizador

In [3]:
def tildes(lista):
    lista = lista.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    return lista

In [4]:
tokens = []

for row in df['texto_busqueda']:
    tokens.append(word_tokenize(row))
    
df['tokens'] = tokens

In [5]:
lista = []
stopwd = stopwords.words('spanish')

for row in df['tokens']:
    temp = [tildes(w.lower()) for w in row if w.lower() not in stopwd if len(w) > 1]
    lista.append(temp)     

## Lista de palabras (uni-gramas)

In [7]:
flatten_corpus = [w for l in lista for w in l ]

## Lista de palabras (n-gramas)

In [9]:
bigram_list = []
trigram_list = []

for i in range(len(lista)):
    bigram_list.append(list(nltk.bigrams(lista[i])))

for i in range(len(lista)):
    trigram_list.append(list(nltk.ngrams(lista[i],3)))

In [10]:
bigram_temp = [w for l in bigram_list for w in l ]
trigram_temp = [str(w) for l in trigram_list for w in l ]
freq_temp = [w for l in trigram_list for w in l]

In [11]:
trigram_corpus = []

for i in trigram_temp:
    temp = i.replace("(", "").replace(")", "").replace(",", "").replace("'", "")
    trigram_corpus.append(temp)

In [12]:
fdist_1 = nltk.FreqDist(flatten_corpus)
fdist_2 = nltk.FreqDist(bigram_temp)
fdist_3 = nltk.FreqDist(freq_temp)
unigram_freq = fdist_1.most_common(15)
bigram_freq = fdist_2.most_common(15)
trigram_freq = fdist_3.most_common(15)

In [13]:
def build_df(data):
    palabras = []
    frecuencia = []
    for w in data:
        palabras.append(w[0])
        frecuencia.append(w[1])
    
    df = pd.DataFrame(frecuencia,palabras)
    return df.reset_index().rename(columns={'index':'Palabra',0:'Frecuencia'})
    
df_freq_uno = build_df(unigram_freq)
df_freq_dos = build_df(bigram_freq)
df_freq_tres = build_df(trigram_freq)

In [25]:
df_freq_tres

,Palabra,Frecuencia
0,"(exoneracion, cuota, manejo)",39
1,"(tc, grupo, uribe)",29
2,"(credito, libre, inversion)",16
3,"(activacion, tarjeta, credito)",15
4,"(bloqueo, alerta, monitoreo)",15
5,"(activar, tarjeta, credito)",15
6,"(cancelacion, tarjeta, credito)",14
7,"(activacion, tc, grupo)",12
8,"(cancelar, tarjeta, credito)",12
9,"(cuota, manejo, tarjeta)",12


In [15]:
path_uno = './texto_uno.txt'
path_dos = './texto_dos.txt'

with open(path_uno,'a+',encoding='utf8') as f:
    for i in flatten_corpus:
        f.write(i)
        f.write(' ')
        
with open(path_dos,'a+',encoding='utf8') as f:
    for i in trigram_corpus:
        f.write(i)
        f.write(' ')

In [16]:
maskArray = np.array(Image.open("cloud.png"))

In [17]:
stop_words_es = set(stopwords.words('spanish'))

In [18]:
nWords = 100

In [19]:
dataset_uno = open("texto_uno.txt", "r", encoding="utf8").read()
dataset_dos = open("texto_dos.txt", "r", encoding="utf8").read()

In [20]:
cloud_uno = WordCloud(background_color = "white", max_words = nWords, mask = maskArray, stopwords = set(stop_words_es))
cloud_dos = WordCloud(background_color = "white", max_words = nWords, mask = maskArray, stopwords = set(stop_words_es))

In [21]:
cloud_uno.generate(dataset_uno)
cloud_dos.generate(dataset_dos)

In [22]:
cloud_uno.to_file("word_cloud_uno.png")
cloud_dos.to_file("word_cloud_dos.png")